Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [61]:
import lab9_lib
from random import choice, randint
from copy import copy
from dataclasses import dataclass
import numpy as np

In [62]:
POPULATION_SIZE = 200
OFFSPRING_SIZE = 300
TOURNAMENT_SIZE = 2
PROBLEM_SIZE = 1
GENOME_SIZE = 1000

In [63]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

In [64]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[int]


def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, GENOME_SIZE - 1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring


def xover(ind1: Individual, ind2: Individual) -> Individual:
    [first, second] = sorted([ind1, ind2], key=lambda i: i.fitness)
    return Individual(
        fitness=None,
        genotype=[
            f if .7 > r else s
            for f, s, r in zip(
                first.genotype,
                second.genotype,
                np.random.random(size=GENOME_SIZE),
            )
        ],
    )

In [65]:
population = [
    Individual(
        genotype=[choice([0, 1]) for _ in range(GENOME_SIZE)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

In [66]:
for generation in range(1000):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        p1 = select_parent(population)
        p2 = select_parent(population)
        o = xover(p1, p2)
        o = mutate(o)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    best_ind = population[0]
    if best_ind.fitness == 1:
        break
    
print(f"Max fitness: {best_ind.fitness:.2%}")
print(f"Number of fitness calls: {fitness.calls}")

Max fitness: 100.00%
Number of fitness calls: 29300
